In [45]:
import os
import json
import re
import regex

In [149]:
article_path = '../articles_2019_2/perevarukha/Perevarukha_tvim-2019.tex'

with open(article_path, 'rt') as f:
    article = f.read()

article

"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n%%%%% Основной файл для авторов журнала ТВИМ\n%%%%% факультет математики и информатики\n%%%%% Таврическая академия Крымского федерального университета им. В.И. Вернадского\n%%%%% Технические редакторы: Анафиев Айдер Сератович, Войтицкий Виктор Иванович\n%%%%% e-mail: tvim@tvim.info\n%%%%% site:   www.tvim.info\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\\documentclass[12pt, a4paper, reqno, intlimits]{amsart}\n\\usepackage[utf8x]{inputenc}\n\\usepackage[ruled]{algorithm}\n%usepackage[noend]{algorithmic}\n\\usepackage{multicol}\n\\usepackage{mathtext}\n\\usepackage[T2A]{fontenc}\n\\usepackage[russian]{babel}\n\\usepackage{amsfonts, amsmath, amsxtra, amsthm, amssymb, latexsym, tvim, mathrsfs, srcltx, dsfont}\n\\usepackage[dvips]{graphicx, epsfig}\n\\usepackage{subfigure}\n\\usepackage{wrapfig}\n\\usepackage{mdframed}\n\\usepackage{wasysym}\n\\usepackage{xcolor}\n\n\\input{__bib_templates__.t

In [47]:
section_pattern = r'\\section\*?\{(?P<title>.*)\}'
title_pattern = r'\\title\{(?P<title>.*)\}'
pattern_textbf = r'\\textbf\{(.*)\}'

In [48]:
def normalize_text(text):
    """
    Удалить: 
        textbf
        textit
    """
    text = re.sub(r'\\textbf|\\textit|\\it|\\bf', '', text) 
    text = re.sub(r'[\{|\}]', '', text) 
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()
    

def extract_title(text):
    """
    Извлечь заголовок.
    """
    title_pattern = r'\\title\{(?P<title>.*)\}'
    m = re.search(title_pattern, text)
    if m:
        title = m['title'] 
        # удалить 'footnote'
        title = re.sub(r'\\footnote\{.*\}', '', title)
        return title

def extract_authors(text):
    """
    Извлечь авторов.
    """
    _authors = []
#     m = re.search(r'\\author\{(.*)\}', text)
    for m1 in re.finditer(r'\\author\{(.*)\}', text):
        authors = m1[1] 
        authors = re.sub(r'\\[;,.:]+', ' ', authors)
        authors = re.sub(r'\s{2,}', ' ', authors)
        name_abbr_pattern = r'(?P<name>[A-ZА-ЯЁa-zа-яё]{1,2}\.)'
        patronymic_abbr_pattern = r'(?P<patronymic>[A-ZА-ЯЁa-zа-яё]{1,2}\.)'
        family_pattern = r'(?P<family>[A-ZА-Яa-zа-яё]{2,})'
        patterns = [r'{}\s*{}\s*{}'.format(name_abbr_pattern, patronymic_abbr_pattern, family_pattern),
                    r'{}\s*{}\s*{}'.format(family_pattern, name_abbr_pattern, patronymic_abbr_pattern)]
        
        for pattern in patterns:
            for m in re.finditer(pattern, authors):
                _authors.append({'family': m['family'],
                                 'name': m['name'],
                                 'patronymic': m['patronymic']})
        return _authors

def extract_ru_abstracts(text):
    """
    Извлечь аннотации.
    """
    # russian
    text = article
    begin_pattern = r'\\begin\{abstract\}.*'
    m = re.search(begin_pattern, text)
    if m:
        p0 = m.end()
        end_pattern = r'\\end\{abstract\}'
        m = re.search(end_pattern, text)
        if m:
            p1 = m.start()
            abstract = text[p0:p1]
            return abstract
    
    
def extract_en_abstracts(text):
    """
    Извлечь аннотации.
    """
    # russian
    text = article
    begin_pattern = r'\\begin\{abstractWithTitle\}.*'
    m = re.search(begin_pattern, text)
    if m:
        p0 = m.end()
        end_pattern = r'\\end\{abstractWithTitle\}'
        m = re.search(end_pattern, text)
        if m:
            p1 = m.start()
            abstract = text[p0:p1]
            return abstract
    

def extract_sections(text):
    """
    Извлечь разделы.
    """
    pattern = r'\\section\*?\{(?P<title>.*)\}'
    sections = []
    for m in re.finditer(pattern, text):
        sections.append(normalize_text(m['title']))
    return sections


def extract_bibliography(text):
    """
    Извлечь список литературы.
    """
    # russian
    text = article
    begin_pattern = r'\\begin\{thebibliography\}.*'
    m = re.search(begin_pattern, text)
    if m:
        p0 = m.end()
        end_pattern = r'\\end\{thebibliography\}'
        m = re.search(end_pattern, text)
        if m:
            p1 = m.start()
            abstract = text[p0:p1]
            return re.sub(r'\%.*', '', abstract)
        
        
def extract_udc(text):
    p = r'(?<=УДК\:)\s*(.*)(?=\})'
    m = re.search(p, text)
    if m:
        return m[1]
    
def extract_msc2010(text):
    p = r'(?<=MSC2010\:)\s*(.*)(?=\})'
    m = re.search(p, text)
    if m:
        return m[1]

In [49]:
text = article
# comment_pattern = r'\%'
# for m in re.finditer(comment_pattern, text):
#     print(m.string)
# extract_bibliography(text)
extract_udc(text)
extract_msc2010(text)

'57M15, 94C15'

In [50]:
extract_authors(article)

[{'family': 'Дубровская', 'name': 'В.', 'patronymic': 'А.'},
 {'family': 'Переварюха', 'name': 'А.', 'patronymic': 'Ю.'}]

In [51]:
for m in re.finditer(section_pattern, article, re.MULTILINE):
    print(m['title'])

\textbf{Введение}
\textbf{Модификация модели Базыкина для критического порога}
\textbf{Функции-регуляторы с запаздыванием в моделях экологии}
\textbf{Характеристики базовых моделей с $N(t-\tau)$}
\textbf{Модификация модели Ферхюльста-Пирла с $\ln(K/N(t-\tau))$}
\textbf{Модификация модели Ферхюльста-Пирла с независимым изъятием}
\textbf{Катастрофический сценарий осциллирующей вспышки}
\textbf{Заключение}


In [52]:
pattern = r'\\[^\{\s]+'
p1 = r'\\textbf'
text = '\\textbf{\\textit{Введение} sjfgklsdfkjg}'
for m in re.finditer(pattern, text, re.MULTILINE):
    print(m[0])

\textbf
\textit


In [53]:
text = '\\textbf{\\textit{Введение} Word1} Word2 {\it Word3 $t-\tau$}'
text = re.sub(r'\\textbf|\\textit|\\it|\\bf', '', text) 
text = re.sub(r'[\{|\}]', '', text) 
text = re.sub(r'\s{2,}', ' ', text)
text

'Введение Word1 Word2 Word3 $t-\tau$'

In [54]:
extract_sections(article)

['Введение',
 'Модификация модели Базыкина для критического порога',
 'Функции-регуляторы с запаздыванием в моделях экологии',
 'Характеристики базовых моделей с $N(t-\\tau)$',
 'Модификация модели Ферхюльста-Пирла с $\\ln(K/N(t-\\tau))$',
 'Модификация модели Ферхюльста-Пирла с независимым изъятием',
 'Катастрофический сценарий осциллирующей вспышки',
 'Заключение']

In [55]:
extract_authors(article)

[{'family': 'Дубровская', 'name': 'В.', 'patronymic': 'А.'},
 {'family': 'Переварюха', 'name': 'А.', 'patronymic': 'Ю.'}]

In [ ]:
# extract english abstract
m = re.search()

In [56]:
title_pattern = r'\\title\{(?P<title>.*)\}'
m = re.search(title_pattern, article)
m

<re.Match object; span=(2694, 2869), match='\\title{МОДЕЛИ СПЕЦИФИЧЕСКИХ ФОРМ БИОЛОГИЧЕСКИХ В>

In [57]:
from TexSoup import TexSoup 

In [58]:
ts = TexSoup(article)

In [15]:
for s in ts.find_all('bibitem'):
    print(s)

\bibitem{ITVIM14}
\bibitem{Myers1997}
\bibitem{Baz1990}
\bibitem{ITVIM15}
\bibitem{Hutchinson1978}
\bibitem{Borz1985}
\bibitem{Frolov2015}
\bibitem{Gopalsamy1990}
\bibitem{Koles2010}
\bibitem{Krebs1974}
\bibitem{Ruan2006}
\bibitem{Perevar2015}


In [ ]:
from docx import Document

In [ ]:
doc = Document('../docs/05predstavlen.docx')

In [ ]:
p = doc.paragraphs[8]

In [ ]:
p.paragraph_format.line_spacing

In [ ]:
new_text = 'научного журнала «Таврический вестник информатики и математики», №2, 2019, 5 статей, 100 страниц.'
pf = p.paragraph_format
doc.paragraphs[8].text = ''
run = p.add_run(new_text)
run.bold = True
run.underline = True
p.paragraph_format.line_spacing = 1.5

In [ ]:
doc.save('../docs/05predstavlen_1.docx')

In [ ]:
p.paragraph_format.line_spacing = 1.5

In [ ]:
with open('resources/__params__.tex', 'rt') as params_file:
    params = params_file.read()

In [ ]:
params

In [ ]:
re.search(r'\\tvimnumber\{(.*)\}', params)

In [ ]:
params = [
    '\\def\\tvimname{Таврический вестник информатики и математики}\n',
    '\\def\\tvimnumber{{№\,{number}\,({total_number})}}\n'.format(number=2, total_number=43),
    '\\def\\tvimyear{{{}}}\n'.format(2019),
    r'\def\tvimemail{article@tvim{.}info}',
    r'\def\tvimwww{www{.}tvim{.}info}',
    '\def\protocolnumber{3}',
    r'\def\protocolday{15}',
    r'\def\protocolmonthname{апреля}',
    r'\def\protocolmonth{04}',
    r'\def\protocolyear{2019}',
    r'\def\protocol{№\,\protocolnumber\ от~\protocolday~\protocolmonthname~\protocolyear\,г.}',
    r'\def\sign2print{\protocolday.\protocolmonth.\protocolyear}',
    r'\def\print_page_count{11}',
    r'\def\tvimissn{ISSN\;1729-3901}',
]

In [ ]:
with open('/Users/ayder/Desktop/temp.tex', 'wt') as f:
    f.writelines(params)

In [ ]:
'{{Number: {number}}}'.format(number=1)

In [ ]:
article

In [ ]:
m_start = re.search(r'\\markboth', article)
m_end = re.search(r'\\end{thebibliography}', article)
print(m_start.start())
print(m_end.end())

In [ ]:
m = re.search(r'\\includegraphics(.*)', article)

In [ ]:
s = article[m.start():m.end()]

In [ ]:
m1 = re.search(r'{(.*)}', s)

In [ ]:
m1[1]

In [ ]:
def update_graphics_path(text, path):
    pos = []
    for m in re.finditer(r'\\includegraphics.*?\{(.+?)\}', text):
        graph_text = text[m.start():m.end()]
        print(graph_text)
        m_image_name = re.search(r'{(.*)}', graph_text)
        pos.append((m.start() + m_image_name.start(), m.start() + m_image_name.end()))
#         new_graph = graph_text[:m_image_name.start()] 
#         text = text[:m.start()] + '{}{{{}}}'.format(new_graph, path + '/' + m_image_name[1]) + text[m.end():]
    d = len(path)+1
    print(pos)
    for i, p in enumerate(pos):
        image_name = text[p[0] + i*d+1:p[1] + i*d-1]
#         print('image_name', image_name)
#         print('{{{}/{}}}'.format(path, image_name))
        text = text[:p[0] + i*d] + '{{{}/{}}}'.format(path, image_name) + text[p[1] + i*d:]
    return text

In [ ]:
article_1 = "afasdf \includegraphics[scale=0.5]{fig1.png} sdfsafaf"
# article_1 = article[24000:26000]
# article_1 = article
update_graphics_path(article_1, 'article/test')

In [ ]:
article_1

In [ ]:
m

In [ ]:
p = r'\\authorInfo\{.*\}?'

In [ ]:
for m in re.finditer(p, article):
    print(a)

In [35]:
from docx import Document

doc = Document('../docs/05predstavlen.docx')

p = doc.paragraphs[8]

new_text = 'научного журнала «Таврический вестник информатики и математики», '\
           '{number}, {year}, {nart} статей, {pagenum} страниц.'.format(number=2, year=2019, nart=2, pagenum=46)
pf = p.paragraph_format
doc.paragraphs[8].text = ''
run = p.add_run(new_text)
run.bold = True
run.underline = True
p.paragraph_format.line_spacing = 1.5
p.alignment = WD_ALIGN_PARAGRAPH.RIGHT

doc.save('../docs/05predstavlen_1.docx')

SyntaxError: invalid syntax (<ipython-input-35-257df150c020>, line 15)

In [32]:
import PyPDF2
reader = PyPDF2.PdfFileReader(open('numbers/tvim_2019_2/tvim_2019_2.pdf', 'rb'))
print(reader.getNumPages())

46


In [ ]:
import os

In [ ]:
os.listdir("numbers/tvim_2019_2")

In [44]:
from docx import Document

doc = Document('../docs/Приложение1_Экспертиза публикации.docx')
for i, p in enumerate(doc.paragraphs):
    print(i, ': ', p.text)

0 :  УТВЕРЖДАЮ
1 :  Проректор по научной деятельности
2 :  ФГАОУВО «Крымский федеральный 
3 :  университет имени  В.И. Вернадского»
4 :  ___________________А. В. Кубышкин
5 :  «_____» _______________  _______
6 :  
7 :  ЭКСПЕРТНОЕ ЗАКЛЮЧЕНИЕ
8 :  о возможности опубликования
9 :  
10 :  Экспертная комиссия (руководитель-эксперт) 	к. т. н., доцент Козлов А. И.
11 :  Таврическая академия
12 :  (структурное  подразделение)
13 :  Федерального государственного бюджетного образовательного учреждения высшего профессионального образования «Крымский Федеральный Университет им. В. И. Вернадского» Министерства образования и науки Российской Федерации, рассмотрев 
14 :  научный журнал «Таврический вестник информатики и математики», 2019, № 1
15 :  (ФИО и должность автора, вид, название материала, темы, статьи, отчета, проекта и т.д.)
16 :  подтверждает, что в представленном материале не содержатся сведения, составляющие государственную тайну, и информация с ограниченным доступом.
17 :  На публикаци

In [186]:
text = article

m = re.search(r'\\begin{thebibliography}', text)
if m:
    bib_begin_pos = m.start()
    
m = re.search(r'\\end{thebibliography}', text)
if m:
    bib_end_pos = m.end()
    
bib_text = text[bib_begin_pos:bib_end_pos]
bib_text = re.sub(r'%\s*(.*)', '', bib_text)
print(bib_text)
bib_text = bib_text.replace('\n', ' ')

bibitems = []
for m in re.finditer(r'\\bibitem{(?P<bibitem>.*?)}', bib_text):
    bibitems.append(m['bibitem'])

for bi in bibitems:
    p = '\\\\bibitem{{{}}}(?P<text>.*?)(?=\\\\bibitem|\\\\end)'.format(bi)
    m = re.search(p, bib_text, re.MULTILINE)
    if m:
        print(m['text'])
        print('-'*80)

\begin{thebibliography}{10}


\bibitem{ITVIM14} 
\tvimRefArticle{Переварюха, А.Ю.}{Разработка вычислительных моделей воспроизводства рыб для сценарного исследования}{}{Таврический вестник информатики и математики}{}{}{2014}{№.\;1}{93--103}

\smallskip
\tvimRefArticleEn{PEREVARYUKHA, A.}{2014}{The development of computational models for fish reproduction in scenario investigations}{Taurida Journal of Computer Science Theory and Mathematics}{1}{}{93-103}

\medskip

\bibitem{Myers1997}
\tvimRefArticleEn{MYERS, R.}{1997}{Why do Fish Stocks Collapse? The Example of Cod in Atlantic Canada}{Ecological Applications}{7}{}{96--106}
\medskip

\bibitem{Baz1990}
\tvimRefArticleEn{BAZYKIN, A.}{1983}{A model of evolutionary appearance of dissipative structure in ecosystems}{J. Math. Biology}{4}{18}{13--23}

\medskip

\bibitem{ITVIM15} 
\tvimRefArticle{Переварюха, А.Ю.}{Метод когнитивной формализации в анализе многофакторной экологической проблемы Каспия}{}{Таврический вестник информатики и математики

In [154]:
text = test_str
regexp = r"\\\\bibitem{ITVIM14}(?P<text>.*?)(?=\\\\bibitem|\\end)"
matches = re.finditer(regexp, text, re.MULTILINE)
for m in matches:
    print(m)

<re.Match object; span=(130, 622), match='\\\\bibitem{ITVIM14} %идентификатор должен быть у>


In [142]:
import re

regex = r"\\\\bibitem{ITVIM14}(?P<text>.*?)(?=\\\\bibitem|\\end)"

test_str = "В следующей работе мы рассмотрим модель вспышки в форме пилообразных осцилляций. \\n\\n\\\\medskip\\n\\\\begin{thebibliography}{10}\\n\\n\\n\\\\bibitem{ITVIM14} %идентификатор должен быть уникальным!!!\\n\\\\tvimRefArticle{Переварюха, А.Ю.}{Разработка вычислительных моделей воспроизводства рыб для сценарного исследования}{}{Таврический вестник информатики и математики}{}{}{2014}{№.\\\\;1}{93--103}\\n\\n\\\\smallskip\\n\\\\tvimRefArticleEn{PEREVARYUKHA, A.}{2014}{The development of computational models for fish reproduction in scenario investigations}{Taurida Journal of Computer Science Theory and Mathematics}{1}{}{93-103}\\n\\n\\\\medskip\\n\\n\\\\bibitem{Myers1997}\\n\\\\tvimRefArticleEn{MYERS, R.}{1997}{Why do Fish Stocks Collapse? The Example of Cod in Atlantic Canada}{Ecological Applications}{7}{}{96--106}\\n\\\\medskip\\n\\n\\\\bibitem{Baz1990}\\n\\\\tvimRefArticleEn{BAZYKIN, A.}{1983}{A model of evolutionary appearance of dissipative structure in ecosystems}{J. Math. Biology}{4}{18}{13--23}\\n\\n\\\\medskip\\n\\n\\\\bibitem{ITVIM15} %идентификатор должен быть уникальным!!!\\n\\\\tvimRefArticle{Переварюха, А.Ю.}{Метод когнитивной формализации в анализе многофакторной экологической проблемы Каспия}{}{Таврический вестник информатики и математики}{}{}{2015}{№.\\\\;4}{52--65}\\n\\n\\\\smallskip\\n\\\\tvimRefArticleEn{PEREVARYUKHA, A.}{2015}{Graph formalization and multifactor analysis of environmental problems on example of the Caspian Sea}{Taurida Journal of Computer Science Theory and Mathematics}{4}{}{52--65}\\n\\\\medskip\\n\\n\\\\bibitem{Hutchinson1978}\\n\\\\tvimRefArticleEn{HUTCHINSON, G.}{1959}{A theoretical ecological model of size distributions among species of animals}{American Naturalist}{4}{93}{117--125}\\n\\\\medskip \\n\\n\\\\bibitem{Borz1985}\\n\\\\tvimRefArticleEn{MUNKEMULLER, T.}{2009}{Hutchinson revisited: patterns of density regulation and the coexistence of strong competitors}{Journal of Theoretical Biology}{1}{259}{109--117}\\n\\\\medskip \\n\\n\\\\bibitem{Frolov2015}\\n\\\\tvimRefArticleEn{FROLOV, A.N.}{2015}{The beet webworm Loxostege sticticalis l. (Lepidoptera, Crambidae) in the focus of agricultural entomology objectives: The periodicity of pest outbreaks}{Entomological Review}{2}{}{147--156}\\n\\\\medskip \\n\\n\\\\bibitem{Gopalsamy1990}\\n \\\\tvimRefArticleEn{GOPALSAMY, K.}{1990}{Global stability in the Delay logistic Equation with discrete delays}{Houston J. Math}{2}{16}{347--356}\\n\\\\medskip \\n\\n\\\\bibitem{Koles2010}\\n\\\\tvimRefArticleEn{KOLESOV, А.}{2010}{A modification of Hutchinson’s equation}{Computational Mathematics and Mathematical Physics}{12}{50}{347--356}\\n\\\\medskip \\n\\\\bibitem{Krebs1974}\\n\\\\tvimRefArticleEn{KREBS, C., MYERS, J.}{1974}{Population Cycles in Small Mammals}{Advances in Ecological Research}{12}{8}{267–-399}\\n\\\\medskip\\n\\n\\\\bibitem{Ruan2006}\\n\\\\tvimRefBookEn{RUAN, S.}{2006}{Delay Differential Equations in Single Species Dynamics}{}{Springer, Berlin.}\\n\\\\medskip\\n\\n\\\\bibitem{Perevar2015}\\n\\\\tvimRefArticleEn{CHEN, C.}{2006}{Modeling and control of local outbreaks of West Nile virus in the United States}{Discrete and Continuous Dynamical Systems}{21}{8}{2423--2449}\\n\\\\medskip\\n\\n\\n\\\\end{thebibliography}\\n\\n\\\\label{Perevaryukha_end}%Обязательная метка конца статьи. Переименуйте ее, указав вместо Yoursurname вашу фамилию!!!\\n\\n\\\\bigskip\\n%%%%%%%%%%%%%%%%%%% Обязательно для заполнения! Печатается в конце журнала!\\n\\\\authorInfo{Переварюха Андрей}{Юрьевич}{к.т.н., с.н.с., лаборатория Прикладной информатики, Санкт-Петербургский институт информатики и автоматизации Российской академии наук, 199178, Россия, г. Санкт-Петербург, ул. 14-линия, 39.}{madelf@rambler.ru}\\n\\n\\\\authorInfo{Дубровская Виктория}{Андреевна}{аспирант, лаборатория Прикладной информатики, Санкт-Петербургский институт информатики и автоматизации Российской академии наук, 199178, Россия, г. Санкт-Петербург, ул. 14-линия, 39.}{torry$\\\\_$2902@mail.ru}\\n\\n\\\\end{document}\\n\\n\""

matches = re.finditer(regex, test_str, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

Match 1 was found at 130-622: \\bibitem{ITVIM14} %идентификатор должен быть уникальным!!!\n\\tvimRefArticle{Переварюха, А.Ю.}{Разработка вычислительных моделей воспроизводства рыб для сценарного исследования}{}{Таврический вестник информатики и математики}{}{}{2014}{№.\\;1}{93--103}\n\n\\smallskip\n\\tvimRefArticleEn{PEREVARYUKHA, A.}{2014}{The development of computational models for fish reproduction in scenario investigations}{Taurida Journal of Computer Science Theory and Mathematics}{1}{}{93-103}\n\n\\medskip\n\n
Group 1 found at 148-622:  %идентификатор должен быть уникальным!!!\n\\tvimRefArticle{Переварюха, А.Ю.}{Разработка вычислительных моделей воспроизводства рыб для сценарного исследования}{}{Таврический вестник информатики и математики}{}{}{2014}{№.\\;1}{93--103}\n\n\\smallskip\n\\tvimRefArticleEn{PEREVARYUKHA, A.}{2014}{The development of computational models for fish reproduction in scenario investigations}{Taurida Journal of Computer Science Theory and Mathematics}{1}{}{9